6 Labels I, O, B, I-V, O-V, B-V

In [1]:
pip install NERDA

     |████████████████████████████████| 3.1 MB 6.0 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
     |████████████████████████████████| 596 kB 37.1 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=2b242494eb17a050cfad9c74d6eb608aaee8063ece9eb5e9e3254427191e16e1
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from collections import defaultdict
import itertools
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

d = defaultdict(list)
d['sentences'] = []
d['tags'] = []
t = defaultdict(list)
t['sentences'] = []
t['tags'] = []
test = defaultdict(list)
test['sentences'] = []
test['tags'] = []


In [5]:
def nerda_format(data):
  ner = defaultdict(list)
  for line1,line2 in itertools.zip_longest(*[data]*2):
    sents = []
    tags = []
    tokens = iter(tk.tokenize(line1))
    current_tag = "O"
    for token in tokens:
      suffix = ""
      if token in line2:
        suffix = "-V" 
                # Good: even if the verb is complex (aka spaced out),suffix can still apply to the entire phrase
                # Bad: false positive -V may occur, if one word that has multiple meanings occurring more than one time in the same sentence 
      try:
        if token == "[":
          current_tag = "I"
          tags.append("B"+suffix)
          sents.append(next(tokens))
        elif token == "]":
          current_tag = "O"
            #   tags.append(current_tag+suffix)
            #   sents.append(next(tokens))
        else:
          tags.append(current_tag+suffix)
          sents.append(token)
      except StopIteration:
        pass
    ner['sentences'].append(sents)
    ner['tags'].append(tags)
  return ner

In [6]:
with open('dev.txt') as data:
    d = nerda_format(data)
    #d['sentences'] = ner['sentences']
    #d['tags'] = ner['tags']

with open('train.txt') as data:
    t = nerda_format(data)
  #  t['sentences'] = ner['sentences']
  #  t['tags'] = ner['tags']

with open('test.txt') as data:
    test = nerda_format(data)
  #  test['sentences'] = ner['sentences']
  #  test['tags'] = ner['tags']

#print(len(d['sentences']))
#print (d['sentences'][1])
#print (d['tags'][1])

In [7]:
d_total_O = 0
t_total_O = 0
test_total_O = 0

d_total_I = 0
t_total_I = 0
test_total_I = 0

d_total_B = 0
t_total_B = 0
test_total_B = 0

d_total_BV = 0
t_total_BV = 0
test_total_BV = 0

d_total_OV = 0
t_total_OV = 0
test_total_OV = 0

d_total_IV = 0
t_total_IV = 0
test_total_IV = 0


for elem in d['tags']:
  d_total_O += elem.count('O')
  d_total_I += elem.count('I')
  d_total_B += elem.count('B')
  d_total_IV += elem.count('I-V')
  d_total_BV += elem.count('B-V')
  d_total_OV += elem.count('O-V')

for elem in t['tags']:
  t_total_O += elem.count('O')
  t_total_I += elem.count('I')
  t_total_B += elem.count('B')
  t_total_IV += elem.count('I-V')
  t_total_BV += elem.count('B-V')
  t_total_OV += elem.count('O-V')

for elem in test['tags']:
  test_total_O += elem.count('O')
  test_total_I += elem.count('I')
  test_total_B += elem.count('B')
  test_total_IV += elem.count('I-V')
  test_total_BV += elem.count('B-V')
  test_total_OV += elem.count('O-V')



In [8]:
print ('I' + '=' + str(d_total_I + t_total_I + test_total_I))
print ('O' + '=' + str(d_total_O + t_total_O + test_total_O))
print ('B' + '=' + str(d_total_B + t_total_B + test_total_B))
print ('IV' + '=' + str(d_total_IV + t_total_IV + test_total_IV))
print ('OV' + '=' + str(d_total_OV + t_total_OV + test_total_OV))
print ('BV' + '=' + str(d_total_BV + t_total_BV + test_total_BV))

I=2133
O=1618
B=207
IV=199
OV=315
BV=1


In [9]:
import NERDA
training = t
validation = d

In [10]:
print(training.get('sentences')[0])

['थः', 'नं', 'छम्ह', 'शक्तिशाली', 'खः', 'धयागु', 'आत्मविश्वास', 'ज्वनाः', 'न्ह्याःवःगु', 'खनेदु']


In [11]:
tag_scheme = [
'B',
'I',
'I-V',
'O-V',
'B-V'
]

In [12]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = training,
dataset_validation = validation,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = 'bert-base-multilingual-cased',
hyperparameters = {'epochs' : 11,
                  'warmup_steps' : 10,
                  'train_batch_size': 7,
                  'learning_rate': 0.0001})

Device automatically set to: cuda


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [13]:
model.train()


 Epoch 1 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:01<00:08,  2.23it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:02<00:07,  2.36it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 1.2666631015864285 Valid Loss = 1.1125197609265645

 Epoch 2 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:01<00:07,  2.44it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:02<00:06,  2.49it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.8687300966544584 Valid Loss = 1.0984989007314045

 Epoch 3 / 11


...

  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:01<00:07,  2.47it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:02<00:06,  2.49it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.13893989405848764 Valid Loss = 1.1770842000842094

 Epoch 11 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:01<00:07,  2.48it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:02<00:06,  2.50it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.12030636502260511 Valid Loss = 1.1722763416667779


'Model trained successfully'

In [14]:
model.evaluate_performance(test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Level,F1-Score,Precision,Recall
0,B,0.888889,0.857143,0.923077
1,I,0.908367,0.876923,0.942149
2,I-V,0.562500,0.562500,0.562500
3,O-V,0.410256,0.347826,0.500000
4,B-V,0.000000,0.000000,0.000000
0,AVG_MICRO,0.819484,NaN,NaN
0,AVG_MICRO,0.554002,NaN,NaN


In [15]:
#text= [['अझ', 'वृहद', 'कथं', 'धायेगु', 'खःसा', 'थ्व', 'विज्ञान', 'ब्रह्माण्ड', 'गथे', 'सनाच्वंगु', 'दु', 'धका', 'स्वयेत','याइगु', 'प्रकृतिया', 'साधारण', 'एनालाइसिस', 'ख']]
print(model.predict([test['sentences'][1]]))
print(test['tags'][1])

[['B', 'I', 'I', 'I-V', 'I', 'I-V', 'I', 'I', 'I-V', 'I-V', 'I', 'I', 'O-V', 'O-V', 'O-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-V']]
['B', 'I', 'I', 'I', 'I', 'I-V', 'I', 'I', 'I-V', 'I-V', 'I', 'I', 'O-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-V', 'O-V', 'O']


In [16]:
## Getting the confusion matrices for every test sentence

from sklearn.metrics import confusion_matrix
y_true = test['tags']
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  print(confusion_matrix(y_true[n], y_pred, labels=["I", "B", "O","I-V","B-V","O-V"]))

[[ 2  0  0  1  0  0]
 [ 0  1  0  0  0  0]
 [ 6  1 13  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  1]]
[[7 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 5 0 0 3]
 [0 0 0 3 0 0]
 [0 0 0 0 0 0]
 [0 0 2 0 0 1]]
[[5 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 1]
 [1 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
[[8 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 8 0 0 2]
 [1 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 0]]
[[13  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0 12  0  0  2]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  2  0  0  0]]
[[4 0 0 0 0 0]
 [1 0 0 0 0 0]
 [2 1 1 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
[[5 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 7 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 2 0 0 0]]
[[8 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 0]]
[[12  0  0  1  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0  3  0  0  1]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  2]]
[[8 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0

In [17]:
# Sum up the 6by6 confusion matrix result in every test sentence
import numpy as np
from sklearn.metrics import confusion_matrix
y_true = test['tags']
d = np.zeros((6,6))
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  d += confusion_matrix(y_true[n], y_pred, labels=["I", "B", "O","B-V","I-V","O-V"])
print(d)

[[114.   0.   0.   0.   7.   0.]
 [  1.  12.   0.   0.   0.   0.]
 [  8.   2.  51.   0.   0.  15.]
 [  0.   0.   0.   0.   0.   0.]
 [  7.   0.   0.   0.   9.   0.]
 [  0.   0.   8.   0.   0.   8.]]
